In [1]:
import pandas as pd
pd.options.display.max_columns = None

In [2]:
df_clinfo = pd.read_csv('https://api.vitaldb.net/cases')
df_clinfo.head()

,caseid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,devices,tracks,casedur,filesize,age,sex,height,weight,bmi,asa,emop,department,optype,dx,opname,approach,position,ane_type,opdur,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,preop_ph,preop_hco3,preop_be,preop_pao2,preop_paco2,preop_spo2,cormack,airway,tubesize,dltubesize,lmasize,iv1,iv2,aline1,aline2,cline1,cline2,intraop_ebl,intraop_uo,intraop_rbc,intraop_ffp,intraop_crystalloid,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,0,11520,-540,10860,-540,10380,-236220,627780,4,78,192,21.3,77.0,M,160.2,67.5,26.3,2.0,N,General surgery,Colorectal,Rectal cancer,Low anterior resection,Open,Lithotomy,General,145,180,8,0,N,Y,N,N,Normal,14.1,189,94.0,33.2,141.0,3.1,134.0,4.3,18,16,10.0,0.82,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Right forearm,N,Left radial,N,N,N,NaN,300.0,0,0,350.0,0,120,0.0,100,70,0,10,0,0,0
1,2,0,15720,-1020,14940,1740,14640,-221160,1506840,5,75,262,26.3,54.0,M,167.3,54.8,19.6,2.0,N,General surgery,Stomach,Advanced gastric cancer,Subtotal gastrectomy,Open,Supine,General,215,245,19,0,N,N,N,N,Normal,10.2,251,110.0,31.9,143.0,4.7,88.0,3.8,18,15,14.0,0.86,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,N,N,N,N,N,50.0,700.0,0,0,800.0,0,150,0.0,0,100,0,20,0,0,0
2,3,0,4440,-540,4260,-540,3960,-218640,40560,5,80,73,6.6,62.0,M,169.1,69.7,24.4,1.0,N,General surgery,Biliary/Pancreas,Gallbladder stone,Cholecystectomy,Videoscopic,Reverse Trendelenburg,General,32,65,2,0,N,N,N,N,Normal,14.2,373,103.0,30.3,144.0,4.9,87.0,4.2,17,34,14.0,1.18,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,N,N,N,N,N,NaN,NaN,0,0,200.0,0,0,0.0,0,50,0,0,0,0,0
3,4,0,21000,-720,20280,2580,18180,-201120,576480,5,74,350,28.3,74.0,M,160.6,53.0,20.5,2.0,N,General surgery,Stomach,Advanced gastric cancer,Distal gastrectomy,Videoscopic,Reverse Trendelenburg,General,255,335,7,1,N,Y,N,N,Normal,14.4,275,103.0,34.5,141.0,4.2,108.0,4.1,23,18,10.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,NaN,NaN,NaN,Left forearm,N,Right radial,N,N,N,NaN,270.0,0,0,2700.0,0,80,0.0,100,100,0,50,0,0,0
4,5,0,21540,-960,22440,2640,21840,-67560,3734040,6,99,359,41.0,66.0,M,171.0,59.7,20.4,3.0,Y,General surgery,Vascular,Aortic aneurysm,Aneurysmal repair,Open,Prone,General,295,350,45,13,N,Y,N,Left anterior fascicular block,Normal,10.1,67,73.0,36.5,146.0,4.4,126.0,2.6,765,77,50.0,4.43,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Right forearm,N,Right radial,N,Right IJV,N,2600.0,1490.0,8,8,7100.0,0,0,0.0,0,160,0,10,900,0,2100


In [3]:
df = pd.read_csv('https://api.vitaldb.net/trks')
df.head()

,caseid,tname,tid
0,1,BIS/BIS,fd869e25ba82a66cc95b38ed47110bf4f14bb368
1,1,BIS/EEG1_WAV,0aa685df768489a18a5e9f53af0d83bf60890c73
2,1,BIS/EEG2_WAV,ad13b2c39b19193c8ae4a2de4f8315f18d61a57e
3,1,BIS/EMG,2525603efe18d982764dbca457affe7a45e766a9
4,1,BIS/SEF,1c91aec859304840dec75acf4a35da78be0e8ef0


In [4]:
df = pd.merge(df, df_clinfo[['caseid', 'opend']], how='left', on='caseid')

In [5]:
df['tname'].value_counts()

Solar8000/HR            6388
Solar8000/PLETH_SPO2    6387
Solar8000/PLETH_HR      6387
Primus/CO2              6363
Primus/PAMB_MBAR        6362
                        ... 
EV1000/SCVO2               1
EV1000/SQI                 1
CardioQ/SVR                1
Vigileo/ART_MBP            1
Solar8000/ST_V5            1
Name: tname, Length: 208, dtype: int64

In [6]:
art_set = set(df[df['tname'].isin(['Solar8000/ART_SBP'])]['caseid'])
nibp_set = set(df[df['tname'].isin(['Solar8000/NIBP_SBP'])]['caseid'])

In [7]:
len(art_set & nibp_set)

3096

### SBP

In [8]:
print('ART_SBP : ', df['tname'].isin(['Solar8000/ART_SBP']).sum())
print('NIBP_SBP : ', df['tname'].isin(['Solar8000/NIBP_SBP']).sum())

ART_SBP :  3725
NIBP_SBP :  5752


* ART와 NIBP가 같이 있는 경우에는 다시 ART를 우선시해서

### MBP

In [9]:
print('ART_MBP : ', df['tname'].isin(['Solar8000/ART_SBP']).sum())
print('NIBP_MBP : ', df['tname'].isin(['Solar8000/NIBP_SBP']).sum())

ART_MBP :  3725
NIBP_MBP :  5752


### DBP

In [10]:
print('ART_DBP : ', df['tname'].isin(['Solar8000/ART_DBP']).sum())
print('NIBP_DBP : ', df['tname'].isin(['Solar8000/NIBP_DBP']).sum())

ART_DBP :  3725
NIBP_DBP :  5752


### HR

In [11]:
print('Solar8000/HR : ', df['tname'].isin(['Solar8000/HR']).sum())
print('Solar8000/PLETH_HR : ', df['tname'].isin(['Solar8000/PLETH_HR']).sum())
print('Vigilance/HR_AVG : ', df['tname'].isin(['Vigilance/HR_AVG']).sum())
print('CardioQ/HR : ', df['tname'].isin(['CardioQ/HR']).sum())

Solar8000/HR :  6388
Solar8000/PLETH_HR :  6387
Vigilance/HR_AVG :  53
CardioQ/HR :  29


### BT

In [12]:
print('Solar8000/BT1 : ', df['tname'].isin(['Solar8000/BT1']).sum())
print('Solar8000/BT2 : ', df['tname'].isin(['Solar8000/BT2']).sum())

Solar8000/BT1 :  5889
Solar8000/BT2 :  864


### SpO2

In [13]:
print('Solar8000/PLETH_SPO2: ', df['tname'].isin(['Solar8000/PLETH_SPO2']).sum())

Solar8000/PLETH_SPO2:  6387


### EtCO2

In [14]:
print('Solar8000/ETCO2: ', df['tname'].isin(['Solar8000/ETCO2']).sum())
print('Primus/ETCO2: ', df['tname'].isin(['Primus/ETCO2']).sum())

Solar8000/ETCO2:  6243
Primus/ETCO2:  6340


### FiO2

In [15]:
print('Solar8000/FIO2: ', df['tname'].isin(['Solar8000/FIO2']).sum())
print('Solar8000/VENT_SET_FIO2: ', df['tname'].isin(['Solar8000/VENT_SET_FIO2']).sum())
print('Primus/FIO2: ', df['tname'].isin(['Primus/FIO2']).sum())
print('Primus/SET_FIO2: ', df['tname'].isin(['Primus/SET_FIO2']).sum())

Solar8000/FIO2:  6240
Solar8000/VENT_SET_FIO2:  2213
Primus/FIO2:  6336
Primus/SET_FIO2:  0


### STII

In [16]:
print('Solar8000/ST_II: ', df['tname'].isin(['Solar8000/ST_II']).sum())

Solar8000/ST_II:  5979


In [17]:
tmp = pd.read_csv('https://api.vitaldb.net/' + '6326f61f2b89f8afb550c102fd1b9c9e44249fe0')
tmp['Time_diff'] = tmp['Time'] - tmp['Time'].tail(1).values[0]

In [18]:
df_clinfo.head()

,caseid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,devices,tracks,casedur,filesize,age,sex,height,weight,bmi,asa,emop,department,optype,dx,opname,approach,position,ane_type,opdur,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,preop_ph,preop_hco3,preop_be,preop_pao2,preop_paco2,preop_spo2,cormack,airway,tubesize,dltubesize,lmasize,iv1,iv2,aline1,aline2,cline1,cline2,intraop_ebl,intraop_uo,intraop_rbc,intraop_ffp,intraop_crystalloid,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,0,11520,-540,10860,-540,10380,-236220,627780,4,78,192,21.3,77.0,M,160.2,67.5,26.3,2.0,N,General surgery,Colorectal,Rectal cancer,Low anterior resection,Open,Lithotomy,General,145,180,8,0,N,Y,N,N,Normal,14.1,189,94.0,33.2,141.0,3.1,134.0,4.3,18,16,10.0,0.82,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Right forearm,N,Left radial,N,N,N,NaN,300.0,0,0,350.0,0,120,0.0,100,70,0,10,0,0,0
1,2,0,15720,-1020,14940,1740,14640,-221160,1506840,5,75,262,26.3,54.0,M,167.3,54.8,19.6,2.0,N,General surgery,Stomach,Advanced gastric cancer,Subtotal gastrectomy,Open,Supine,General,215,245,19,0,N,N,N,N,Normal,10.2,251,110.0,31.9,143.0,4.7,88.0,3.8,18,15,14.0,0.86,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,N,N,N,N,N,50.0,700.0,0,0,800.0,0,150,0.0,0,100,0,20,0,0,0
2,3,0,4440,-540,4260,-540,3960,-218640,40560,5,80,73,6.6,62.0,M,169.1,69.7,24.4,1.0,N,General surgery,Biliary/Pancreas,Gallbladder stone,Cholecystectomy,Videoscopic,Reverse Trendelenburg,General,32,65,2,0,N,N,N,N,Normal,14.2,373,103.0,30.3,144.0,4.9,87.0,4.2,17,34,14.0,1.18,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Left forearm,N,N,N,N,N,NaN,NaN,0,0,200.0,0,0,0.0,0,50,0,0,0,0,0
3,4,0,21000,-720,20280,2580,18180,-201120,576480,5,74,350,28.3,74.0,M,160.6,53.0,20.5,2.0,N,General surgery,Stomach,Advanced gastric cancer,Distal gastrectomy,Videoscopic,Reverse Trendelenburg,General,255,335,7,1,N,Y,N,N,Normal,14.4,275,103.0,34.5,141.0,4.2,108.0,4.1,23,18,10.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,NaN,NaN,NaN,Left forearm,N,Right radial,N,N,N,NaN,270.0,0,0,2700.0,0,80,0.0,100,100,0,50,0,0,0
4,5,0,21540,-960,22440,2640,21840,-67560,3734040,6,99,359,41.0,66.0,M,171.0,59.7,20.4,3.0,Y,General surgery,Vascular,Aortic aneurysm,Aneurysmal repair,Open,Prone,General,295,350,45,13,N,Y,N,Left anterior fascicular block,Normal,10.1,67,73.0,36.5,146.0,4.4,126.0,2.6,765,77,50.0,4.43,NaN,NaN,NaN,NaN,NaN,NaN,I,Oral,7.5,NaN,NaN,Right forearm,N,Right radial,N,Right IJV,N,2600.0,1490.0,8,8,7100.0,0,0,0.0,0,160,0,10,900,0,2100


## Get Data

In [19]:
def load_data(data, track_nm):
    tmp = pd.read_csv('https://api.vitaldb.net/' + data['tid'].values[0])
    tmp['Time_diff'] = tmp['Time'] - data['opend'].values[0]
    return pd.DataFrame([tmp[(tmp['Time_diff'] >= (-60*30)) & (tmp['Time_diff'] < 0)][track_nm].mean(), tmp[(tmp['Time_diff'] >= (-60*30)) & (tmp['Time_diff'] < 0)][track_nm].std()])

#### for loop..
from tqdm.notebook import tqdm
tqdm.pandas()
df[df['tname'].isin(['Solar8000/BT1'])][0:10].groupby(['caseid']).progress_apply(lambda x: load_data(x, 'Solar8000/BT1')).reset_index().rename(columns={'level_1':'cal_type',0:'Value'})

In [20]:
track_list = [   
              ## HR
              'Solar8000/HR', 
              ## BT
              'Solar8000/BT1', 
              ## SPO2
              'Solar8000/PLETH_SPO2',
              ## FIO2
              'Solar8000/FIO2', 
              ## ST_II
              'Solar8000/ST_II', 
              ## ETCO2
              'Solar8000/ETCO2', 
              ## SBP
              'Solar8000/ART_SBP', 'Solar8000/NIBP_SBP',
              ## DBP
              'Solar8000/NIBP_DBP', 'Solar8000/ART_DBP',
              ## MBP
              'Solar8000/NIBP_MBP', 'Solar8000/ART_MBP'
             ]

In [21]:
from tqdm.notebook import tqdm

for track_nm in track_list:
    tqdm.pandas()
    df[df['tname'].isin([track_nm])].groupby(['caseid'])\
    .progress_apply(lambda x: load_data(data=x,track_nm=track_nm))\
    .reset_index().rename(columns={'level_1':'cal_type',0:'Value'})\
    .to_csv('./{}.csv'.format(track_nm.split('/')[1]), index=False)

/home/khl256/anaconda3/envs/deeplearning/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


tmp = pd.DataFrame()
for track_nm in track_list:
    track_nm = track_nm.split('/')[1]
    df_tmp = pd.read_csv('./{}.csv'.format(track_nm))
    tmp = pd.pivot_table(data=df_tmp, values='Value', index='caseid', columns='cal_type')
    tmp.columns.name = None
    tmp.reset_index().rename(columns={0:'mean_{}'.format(track_nm),1:'std_{}'.format(track_nm)}).to_csv('./{}_pivot.csv'.format(track_nm), index=False)

In [22]:
tmp = pd.DataFrame()
for i, track_nm in enumerate(track_list):
    track_nm = track_nm.split('/')[1]
    df_tmp = pd.read_csv('./{}.csv'.format(track_nm))
    df_tmp = pd.pivot_table(data=df_tmp, values='Value', index='caseid', columns='cal_type')
    df_tmp.columns.name = None
    df_tmp = df_tmp.reset_index().rename(columns={0:'mean_{}'.format(track_nm),1:'std_{}'.format(track_nm)})
    if i == 0:
        tmp = tmp.append(df_tmp)
    else:
        tmp = pd.merge(tmp, df_tmp, how='outer')

In [23]:
tmp.head()

,caseid,mean_HR,std_HR,mean_BT1,std_BT1,mean_PLETH_SPO2,std_PLETH_SPO2,mean_FIO2,std_FIO2,mean_ST_II,std_ST_II,mean_ETCO2,std_ETCO2,mean_ART_SBP,std_ART_SBP,mean_NIBP_SBP,std_NIBP_SBP,mean_NIBP_DBP,std_NIBP_DBP,mean_ART_DBP,std_ART_DBP,mean_NIBP_MBP,std_NIBP_MBP,mean_ART_MBP,std_ART_MBP
0,1,81.485556,2.996626,36.116111,0.093483,99.055556,0.757038,47.153333,11.716376,-0.519222,0.092866,33.675556,0.678005,116.444444,19.841312,NaN,NaN,NaN,NaN,51.573333,9.962676,NaN,NaN,75.945556,13.616238
1,2,79.102336,1.852761,35.730145,0.045914,100.000000,0.000000,38.688543,13.763545,NaN,NaN,36.868743,2.527469,NaN,NaN,103.014461,4.396903,65.304783,2.581088,NaN,NaN,76.676307,3.967126,NaN,NaN
2,3,75.496940,7.881960,35.472559,0.057151,99.414663,1.648374,58.315927,28.819101,0.954182,0.183815,34.464752,2.591728,-0.857143,0.363137,107.215422,7.365301,63.503060,5.052693,-0.857143,0.363137,80.075887,4.701024,-0.857143,0.363137
3,4,85.750562,6.536445,35.349480,0.050034,100.000000,0.000000,48.066292,24.410602,-0.586573,0.193934,30.343820,4.832957,111.813483,20.446562,NaN,NaN,NaN,NaN,51.089888,6.517331,NaN,NaN,70.892135,10.227758
4,5,83.275785,3.881121,29.771968,0.225909,99.790850,0.717713,83.005282,5.320253,0.318468,0.223069,39.570423,2.281427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
bp_list = ['DBP', 'SBP', 'MBP']
for bp_type in bp_list:
    tmp['mean_' + bp_type] = tmp['mean_ART_' + bp_type]
    tmp.loc[tmp['mean_' + bp_type].isnull(), 'mean_' + bp_type] = tmp.loc[tmp['mean_' + bp_type].isnull(), 'mean_NIBP_' + bp_type]
    tmp['std_' + bp_type] = tmp['std_ART_' + bp_type]
    tmp.loc[tmp['std_' + bp_type].isnull(), 'std_' + bp_type] = tmp.loc[tmp['std_' + bp_type].isnull(), 'std_NIBP_' + bp_type]

In [25]:
#tmp['mean_DBP'] = tmp['mean_ART_DBP']
#tmp.loc[tmp['mean_DBP'].isnull(), 'mean_DBP'] = tmp.loc[tmp['mean_DBP'].isnull(), 'mean_NIBP_DBP']
#tmp['std_DBP'] = tmp['std_ART_DBP']
#tmp.loc[tmp['std_DBP'].isnull(), 'std_DBP'] = tmp.loc[tmp['std_DBP'].isnull(), 'std_NIBP_DBP']

#tmp['mean_MBP'] = tmp['mean_ART_MBP']
#tmp.loc[tmp['mean_MBP'].isnull(), 'mean_MBP'] = tmp.loc[tmp['mean_MBP'].isnull(), 'mean_NIBP_MBP']
#tmp['std_MBP'] = tmp['std_ART_MBP']
#tmp.loc[tmp['std_MBP'].isnull(), 'std_MBP'] = tmp.loc[tmp['std_MBP'].isnull(), 'std_NIBP_MBP']

#tmp['mean_SBP'] = tmp['mean_ART_SBP']
#tmp.loc[tmp['mean_SBP'].isnull(), 'mean_SBP'] = tmp.loc[tmp['mean_SBP'].isnull(), 'mean_NIBP_SBP']
#tmp['std_SBP'] = tmp['std_ART_SBP']
#tmp.loc[tmp['std_SBP'].isnull(), 'std_SBP'] = tmp.loc[tmp['std_SBP'].isnull(), 'std_NIBP_SBP']

In [26]:
tmp.to_csv('./data_intraop.csv', index=False)

In [27]:
df = pd.read_csv('./data_preop_preprocessed.csv')
df.head()

,caseid,age,sex,bmi,asa,emop,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,intraop_epi,intraop_ca,WBC,Hb,PLT,Albumin,GOT (AST),GPT (ALT),Creatinine,Sodium,Potassium,Chloride,hs-CRP quantitation,PT (INR),aPTT,Fibrinogen,pH,pCO₂,pO₂,HCO3-,BE,Calcium,Glucose,Lactic acid
0,1,77.0,1,26.3,2.0,0,180,8,0,0,1,0,0,Normal,14.1,189.0,94.0,33.2,141.0,3.1,134.0,4.3,18.0,16.0,10.0,0.82,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.0,3.1,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,54.0,1,19.6,2.0,0,245,19,0,0,0,0,0,Normal,10.2,251.0,110.0,31.9,143.0,4.7,88.0,3.8,18.0,15.0,14.0,0.86,0,0,4.36,10.2,251.0,NaN,NaN,NaN,NaN,143.0,4.7,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,62.0,1,24.4,1.0,0,65,2,0,0,0,0,0,Normal,14.2,373.0,103.0,30.3,144.0,4.9,87.0,4.2,17.0,34.0,14.0,1.18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,74.0,1,20.5,2.0,0,335,7,1,0,1,0,0,Normal,14.4,275.0,103.0,34.5,141.0,4.2,108.0,4.1,23.0,18.0,10.0,0.96,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,4.3,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,66.0,1,20.4,3.0,1,350,45,13,0,1,0,1,Normal,10.1,67.0,73.0,36.5,146.0,4.4,126.0,2.6,765.0,77.0,50.0,4.43,0,2100,14.64,15.3,158.0,4.6,64.0,30.0,1.83,139.0,4.9,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.9,140.0,NaN


In [28]:
df_final = pd.merge(df, tmp, how='left', on='caseid')
df_final.head()

,caseid,age,sex,bmi,asa,emop,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,intraop_epi,intraop_ca,WBC,Hb,PLT,Albumin,GOT (AST),GPT (ALT),Creatinine,Sodium,Potassium,Chloride,hs-CRP quantitation,PT (INR),aPTT,Fibrinogen,pH,pCO₂,pO₂,HCO3-,BE,Calcium,Glucose,Lactic acid,mean_HR,std_HR,mean_BT1,std_BT1,mean_PLETH_SPO2,std_PLETH_SPO2,mean_FIO2,std_FIO2,mean_ST_II,std_ST_II,mean_ETCO2,std_ETCO2,mean_ART_SBP,std_ART_SBP,mean_NIBP_SBP,std_NIBP_SBP,mean_NIBP_DBP,std_NIBP_DBP,mean_ART_DBP,std_ART_DBP,mean_NIBP_MBP,std_NIBP_MBP,mean_ART_MBP,std_ART_MBP,mean_DBP,std_DBP,mean_SBP,std_SBP,mean_MBP,std_MBP
0,1,77.0,1,26.3,2.0,0,180,8,0,0,1,0,0,Normal,14.1,189.0,94.0,33.2,141.0,3.1,134.0,4.3,18.0,16.0,10.0,0.82,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.0,3.1,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.485556,2.996626,36.116111,0.093483,99.055556,0.757038,47.153333,11.716376,-0.519222,0.092866,33.675556,0.678005,116.444444,19.841312,NaN,NaN,NaN,NaN,51.573333,9.962676,NaN,NaN,75.945556,13.616238,51.573333,9.962676,116.444444,19.841312,75.945556,13.616238
1,2,54.0,1,19.6,2.0,0,245,19,0,0,0,0,0,Normal,10.2,251.0,110.0,31.9,143.0,4.7,88.0,3.8,18.0,15.0,14.0,0.86,0,0,4.36,10.2,251.0,NaN,NaN,NaN,NaN,143.0,4.7,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.102336,1.852761,35.730145,0.045914,100.000000,0.000000,38.688543,13.763545,NaN,NaN,36.868743,2.527469,NaN,NaN,103.014461,4.396903,65.304783,2.581088,NaN,NaN,76.676307,3.967126,NaN,NaN,65.304783,2.581088,103.014461,4.396903,76.676307,3.967126
2,3,62.0,1,24.4,1.0,0,65,2,0,0,0,0,0,Normal,14.2,373.0,103.0,30.3,144.0,4.9,87.0,4.2,17.0,34.0,14.0,1.18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.496940,7.881960,35.472559,0.057151,99.414663,1.648374,58.315927,28.819101,0.954182,0.183815,34.464752,2.591728,-0.857143,0.363137,107.215422,7.365301,63.503060,5.052693,-0.857143,0.363137,80.075887,4.701024,-0.857143,0.363137,-0.857143,0.363137,-0.857143,0.363137,-0.857143,0.363137
3,4,74.0,1,20.5,2.0,0,335,7,1,0,1,0,0,Normal,14.4,275.0,103.0,34.5,141.0,4.2,108.0,4.1,23.0,18.0,10.0,0.96,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,4.3,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.750562,6.536445,35.349480,0.050034,100.000000,0.000000,48.066292,24.410602,-0.586573,0.193934,30.343820,4.832957,111.813483,20.446562,NaN,NaN,NaN,NaN,51.089888,6.517331,NaN,NaN,70.892135,10.227758,51.089888,6.517331,111.813483,20.446562,70.892135,10.227758
4,5,66.0,1,20.4,3.0,1,350,45,13,0,1,0,1,Normal,10.1,67.0,73.0,36.5,146.0,4.4,126.0,2.6,765.0,77.0,50.0,4.43,0,2100,14.64,15.3,158.0,4.6,64.0,30.0,1.83,139.0,4.9,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.9,140.0,NaN,83.275785,3.881121,29.771968,0.225909,99.790850,0.717713,83.005282,5.320253,0.318468,0.223069,39.570423,2.281427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
import numpy as np
df_final['preop_pft'] = np.where(df_final['preop_pft'].str.contains('Normal'), 0, 1)

In [30]:
df_final.to_csv('./data_preprocessed.csv', index=False)

In [31]:
df_final

,caseid,age,sex,bmi,asa,emop,anedur,los_postop,los_icu,death_inhosp,preop_htn,preop_dm,preop_arrhythmia,preop_pft,preop_hb,preop_plt,preop_pt,preop_aptt,preop_na,preop_k,preop_glucose,preop_alb,preop_got,preop_gpt,preop_bun,preop_cr,intraop_epi,intraop_ca,WBC,Hb,PLT,Albumin,GOT (AST),GPT (ALT),Creatinine,Sodium,Potassium,Chloride,hs-CRP quantitation,PT (INR),aPTT,Fibrinogen,pH,pCO₂,pO₂,HCO3-,BE,Calcium,Glucose,Lactic acid,mean_HR,std_HR,mean_BT1,std_BT1,mean_PLETH_SPO2,std_PLETH_SPO2,mean_FIO2,std_FIO2,mean_ST_II,std_ST_II,mean_ETCO2,std_ETCO2,mean_ART_SBP,std_ART_SBP,mean_NIBP_SBP,std_NIBP_SBP,mean_NIBP_DBP,std_NIBP_DBP,mean_ART_DBP,std_ART_DBP,mean_NIBP_MBP,std_NIBP_MBP,mean_ART_MBP,std_ART_MBP,mean_DBP,std_DBP,mean_SBP,std_SBP,mean_MBP,std_MBP
0,1,77.0,1,26.3,2.0,0,180,8,0,0,1,0,0,0,14.1,189.0,94.0,33.2,141.0,3.1,134.0,4.3,18.0,16.0,10.0,0.82,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.0,3.1,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.485556,2.996626,36.116111,9.348278e-02,99.055556,0.757038,47.153333,11.716376,-0.519222,0.092866,33.675556,0.678005,116.444444,19.841312,NaN,NaN,NaN,NaN,51.573333,9.962676,NaN,NaN,75.945556,13.616238,51.573333,9.962676,116.444444,19.841312,75.945556,13.616238
1,2,54.0,1,19.6,2.0,0,245,19,0,0,0,0,0,0,10.2,251.0,110.0,31.9,143.0,4.7,88.0,3.8,18.0,15.0,14.0,0.86,0,0,4.36,10.2,251.0,NaN,NaN,NaN,NaN,143.0,4.7,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.102336,1.852761,35.730145,4.591414e-02,100.000000,0.000000,38.688543,13.763545,NaN,NaN,36.868743,2.527469,NaN,NaN,103.014461,4.396903,65.304783,2.581088,NaN,NaN,76.676307,3.967126,NaN,NaN,65.304783,2.581088,103.014461,4.396903,76.676307,3.967126
2,3,62.0,1,24.4,1.0,0,65,2,0,0,0,0,0,0,14.2,373.0,103.0,30.3,144.0,4.9,87.0,4.2,17.0,34.0,14.0,1.18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.496940,7.881960,35.472559,5.715053e-02,99.414663,1.648374,58.315927,28.819101,0.954182,0.183815,34.464752,2.591728,-0.857143,0.363137,107.215422,7.365301,63.503060,5.052693,-0.857143,0.363137,80.075887,4.701024,-0.857143,0.363137,-0.857143,0.363137,-0.857143,0.363137,-0.857143,0.363137
3,4,74.0,1,20.5,2.0,0,335,7,1,0,1,0,0,0,14.4,275.0,103.0,34.5,141.0,4.2,108.0,4.1,23.0,18.0,10.0,0.96,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,4.3,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.750562,6.536445,35.349480,5.003448e-02,100.000000,0.000000,48.066292,24.410602,-0.586573,0.193934,30.343820,4.832957,111.813483,20.446562,NaN,NaN,NaN,NaN,51.089888,6.517331,NaN,NaN,70.892135,10.227758,51.089888,6.517331,111.813483,20.446562,70.892135,10.227758
4,5,66.0,1,20.4,3.0,1,350,45,13,0,1,0,1,0,10.1,67.0,73.0,36.5,146.0,4.4,126.0,2.6,765.0,77.0,50.0,4.43,0,2100,14.64,15.3,158.0,4.6,64.0,30.0,1.83,139.0,4.9,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.9,140.0,NaN,83.275785,3.881121,29.771968,2.259093e-01,99.790850,0.717713,83.005282,5.320253,0.318468,0.223069,39.570423,2.281427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,6384,64.0,1,24.2,1.0,0,245,8,0,0,0,0,0,0,14.5,279.0,109.0,31.0,NaN,NaN,100.0,4.2,31.0,33.0,13.0,0.99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143.0,4.2,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.584821,7.463689,35.272763,1.696362e+00,98.829436,4.181958,63.870000,22.223638,0.283751,0.121656,33.507143,5.832542,NaN,NaN,148.035139,17.005569,87.961933,13.128566,NaN,NaN,109.765739,21.906309,NaN,NaN,87.961933,13.128566,148.035139,17.005569,109.765739,21.906309
6384,6385,69.0,1,24.6,2.0,0,335,20,0,0,1,0,0,0,15.2,239.0,114.0,28.7,144.0,4.0,140.0,3.7,18.0,28.0,19.0,0.84,0,300,5.45,15.2,239.0,3.7,18.0,28.0,0.84,143.0,3.6,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N